In [ ]:
## pip install -q ultralytics
## The docs: https://docs.ultralytics.com/fr/modes/

In [ ]:
from ultralytics import YOLO
import cv2 
import json

train = False

In [ ]:
if train == True:
    # Load a pretrained YOLO model (recommended for training)
    model = YOLO("yolo11n.pt")
    model.train(data="room_segmentation/data.yaml", epochs=200, imgsz=640, plots=True)

In [ ]:
# Load the dectection YOLO trained model
model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
# Run batched inference on a list of images
results = model(['test_image/samples/output0.png'])  # return a list of Results objects

# Process results list
for i, result in enumerate(results):
    boxes = result.boxes  # Boxes object for bounding box outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    # result.show()  # display to screen
    result.save(filename=f"result{i}.png")  # save to disk

In [ ]:
def customize_plot(orig_pic, result, threshold_conf=None):
    boxes = result.boxes  # Boxes object for bounding box outputs
    xyxys = boxes.xyxy
    confs = boxes.conf
    for i, xyxy in enumerate(xyxys):
        if threshold_conf is not None and confs[i] < threshold_conf:
                continue
        x1, y1, x2, y2 = map(int, xyxy)
        cv2.rectangle(orig_pic, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw rectangle
    return orig_pic     

In [ ]:
# Load image
image = cv2.imread('test_image/samples/output0.png')
inferance_image = customize_plot(image, results[0])
cv2.imwrite("test_image/results_samples/output0.png", inferance_image)

In [ ]:
def save_room_geojson(paths, results, threshold_conf=None):
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }
    
    for i, result in enumerate(results):
        boxes = result.boxes  # Bounding box outputs
        xyxys = boxes.xyxy
        confs = boxes.conf
        for i, xyxy in enumerate(xyxys):
            # Optionally filter by confidence
            if threshold_conf is not None and confs[i] < threshold_conf:
                continue
            # Create GeoJSON feature for the bounding box
            x1, y1, x2, y2 = map(int, xyxy)
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [[
                        [x1, y1],  # Top-left
                        [x2, y1],  # Top-right
                        [x2, y2],  # Bottom-right
                        [x1, y2],  # Bottom-left
                        [x1, y1]   # Closing the polygon
                    ]]
                },
                "properties": {
                    "confidence": float(confs[i])
                }
            }
            geojson["features"].append(feature)
        # Save the GeoJSON to a file
        print(i)
        with open(paths[i], 'w') as f:
            json.dump(geojson, f, indent=4)
    print(f"GeoJSON saved")


In [ ]:
paths = ['test_image/geosjson/output0.geojson']
save_room_geojson(paths, results)